In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
df = pd.DataFrame()
print(df)
pages = [str(i) for i in range(1,3)]
country_release=[]
_language = []

Empty DataFrame
Columns: []
Index: []


In [3]:
#Extracting information from multiple pages 

for page in pages:
  print("Importing page", page)
  url = "https://musicbrainz.org/search?query=drake&type=release&limit=100&method=indexed&page=" + page 
  response=requests.get(url)
  print(response)
  sleep(randint(8,15))
  #clear_output(wait = True)

  soup = BeautifulSoup(response.text, 'html.parser')
  tbl=soup.find('table',{'class': 'tbl'})

  df_new = pd.read_html(str(tbl))[0]
  df = df.append(df_new)

  rows = tbl.find_all('tr')

#get the country and date full description instead of taking the abbrviation using the title attribute
  for row in range(1, len(rows)):
    td = rows[row].find_all('td')
    if len(td[4].find_all('li', {'class':'show-all'})) > 0:
      country_release.append("Multiple event releases")
    else:
      li = td[4].find_all('li', {'class':'release-event'})
      if len(li) > 0:
        country = li[0].find_all('abbr')
        if len(country) > 0:
          span = li[0].find_all('span', {'class':'release-date'})
          release_date = span[0].text
          attribute = country[0].attrs
          title = attribute.get('title')
          country_release.append(str(title)+str("/")+str(release_date))
        else:
          country_release.append(None)
      else:
        country_release.append(None)
      
#get the language full name instead of taking the abbrviation using the title attribute 
    languages = td[8].find_all('abbr')
    if len(languages) > 1:
      lang1 = languages[0].get("title")
      lang2 = languages[1].get("title")
      _language.append(str(lang1) + str("/") + str(lang2))
    elif len(languages) == 1:
      _language.append(languages[0].get("title"))
    else:
      _language.append(None)

Importing page 1
<Response [200]>
Importing page 2
<Response [200]>


In [4]:
#update the values in the dataframe
df["Country/Date"] = country_release
df["Language"] = _language

In [5]:
df.shape

(200, 11)

In [22]:
df.tail()

,Name,Artist,Format,Tracks,Language,Type,Status,Country,Date
95,Forever (Tyler Blue drumstep remix),Drake,Digital Media,14,English/Latin,Single + Remix,Promotion,[Worldwide],2011-09-28
96,The Drizzy Effect,Drake,Digital Media,13,English/Latin,Album + Mixtape/Street,Bootleg,[Worldwide],2009-01-01
97,5AM in Toronto,Drake,Digital Media,14,English/Latin,Single,Official,[Worldwide],2013-03-07
98,Find Your Love,Drake,Digital Media,1,English/Latin,Single,Official,[Worldwide],2010-05-14
99,9AM in Dallas,Drake,Digital Media,10,English/Latin,Single,Official,[Worldwide],2010-06-15


Split the Country/Date Column into two separate columns

In [7]:
df[['Country', 'Date']] = df['Country/Date'].str.split('/', expand=True)
df = df.drop(['Country/Date'], axis=1)
df

Checking Missing value 

In [9]:
# Checking missing values
print(df.isnull().sum())

Name          0
Artist        0
Format        0
Tracks        0
Label        58
Catalog#    116
Barcode     104
Language     11
Type          3
Status       12
Country      24
Date         26
dtype: int64


Drop columns with maximum missing values and least useful 

In [10]:
print("Columns Catalog#, label  and Barcode has many missing values and not adding much value to the dataset. \nHence, will be dropped from the dataset.")
df = df.drop(['Label', 'Catalog#', 'Barcode'], axis=1)

Columns Catalog#, label  and Barcode has many missing values and not adding much value to the dataset. 
Hence, will be dropped from the dataset.


Removing rows with missing values 

In [11]:
print("Other columns have minimum number of rows with NULL values.\nRows with NULL values will be removed.")
df = df.dropna()

Other columns have minimum number of rows with NULL values.
Rows with NULL values will be removed.


Sorting the columns

In [12]:
df.loc[:,['Name', 'Artist', 'Format','Tracks', 'Country', 'Date', 'Language', 'Type', 'Status']]
df.head()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160 entries, 0 to 99
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      160 non-null    object
 1   Artist    160 non-null    object
 2   Format    160 non-null    object
 3   Tracks    160 non-null    object
 4   Language  160 non-null    object
 5   Type      160 non-null    object
 6   Status    160 non-null    object
 7   Country   160 non-null    object
 8   Date      160 non-null    object
dtypes: object(9)
memory usage: 12.5+ KB


Deriving statistical information for each column

In [14]:
df.describe()

,Name,Artist,Format,Tracks,Language,Type,Status,Country,Date
count,160,160,160,160,160,160,160,160,160
unique,129,98,16,34,14,19,4,18,133
top,Mandrake,Drake,Digital Media,1,English/Latin,Album,Official,[Worldwide],2004
freq,9,18,66,26,131,64,141,52,6


Removing duplicate rows 

In [15]:
#counting duplicate rows in the dataframe
print("The dataset has", df.shape[0], "rows and", df.shape[1], "columns.")
dupRows = len(df)-len(df.drop_duplicates())
print("Out of the total,", dupRows, "duplicate records will be dropped.")

#Dropping Duplicate Rows 
df=df.drop_duplicates()
print("Dataset shape after removing duplicates is:")
df.shape

The dataset has 160 rows and 9 columns.
Out of the total, 0 duplicate records will be dropped.
Dataset shape after removing duplicates is:


(160, 9)

In the column Tracks, there are values like "10 + 14 + 18 + 20". So added a split function that splits the string and adds the number together and saves the total back in the dataframe.

In [16]:
def fnSplit(text):
  total = 0
  text = text.split("+")
  for i in text:
    total += int(i)
  return total

for index, row in df.iterrows():
    df.loc[index,'Tracks'] = fnSplit(str(row["Tracks"]))

The Date column had dates in various formats, converted them all into one single format using the below command.

In [17]:
# convert the 'Date' column to datetime format so all date formats are the same
df['Date']= pd.to_datetime(df['Date'])

Removing leading and trailing white spaces from all columns 

In [18]:
df['Name'] = df['Name'].str.strip()
df['Artist'] = df['Artist'].str.strip()
df['Format'] = df['Format'].str.strip()
df['Type'] = df['Type'].str.strip()
df['Status'] = df['Status'].str.strip()

In [21]:
#Saving data to a csv files
df.to_csv('music_releases.csv')

Inserting the data into a dataset
